In [ ]:
#!ls /opt/conda/lib/libcud*

In [ ]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/opt/conda/lib/libcudart.so

In [ ]:
!ls /opt/conda/lib/python3.10/site-packages/bitsandbytes/

In [ ]:
!cp /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
#!pip install -r requirements_1.txt

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
import os
os.environ['LD_LIBRARY_PATH'] = '/opt/conda/lib/'

In [ ]:
#!pip uninstall -y torch

In [ ]:
#!pip uninstall -y torchaudio

In [ ]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Version Check

In [ ]:
#!pip install --upgrade google-api-python-client


In [ ]:

import IPython
import torch
import torchaudio
import transformers

In [ ]:
print (f"{torch.__version__ = }")

In [ ]:
torchaudio.__version__

In [ ]:
print (f"{transformers.__version__ = }")

In [ ]:
print (f"{torch.cuda.is_available() = }")


In [ ]:
print (f"{torch.version.cuda = }")

In [ ]:
print (f"{torch.cuda.get_device_name(0) = }")

In [ ]:
assert torch.cuda.is_available()

In [ ]:
#!pip install datasets

In [ ]:
#!pip install soundfile

In [ ]:
#!pip install librosa

In [ ]:
#!pip install evaluate jiwer


In [ ]:

import numpy as np
from copy import copy
from typing import List
import IPython
import transformers
from datetime import datetime
from numpy.random import random
from color import Color

#import torchaudio
#import IPython.display as ipd

from sentiment import Sentiment, SentimentScore
from conversation import Conversation

In [ ]:

import soundfile as sf
import evaluate
import numpy as np

from datasets import load_dataset
from datasets import Audio

from transformers import pipeline
from transformers import AutoFeatureExtractor
from transformers import Speech2Text2Processor
from transformers import SpeechEncoderDecoderModel
from transformers import Speech2TextForConditionalGeneration



In [ ]:
model_name = "facebook/s2t-wav2vec2-large-en-de"
model = Speech2TextForConditionalGeneration.from_pretrained(model_name)

In [ ]:
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")


In [ ]:
def map_to_array(batch):
    speech, _ = sf.read(batch["path"])
    batch["speech"] = speech
    return batch

In [ ]:
minds = load_dataset("PolyAI/minds14", name="en-US", split="train")

In [ ]:
minds

In [ ]:
ds = minds.map(map_to_array)

In [ ]:
ds = ds.remove_columns(["path", "transcription", "english_transcription", "lang_id"])


In [ ]:
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))


In [ ]:
ds = ds.rename_column("intent_class", "label")


In [ ]:
labels = ds.features["label"].names


In [ ]:
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
id2label

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)


In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [ ]:
ds_T = ds.map(preprocess_function, remove_columns="audio", batched=True)


In [ ]:



accuracy = evaluate.load("accuracy")


In [ ]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)


In [ ]:
audio_file = ds[0]["audio"]["path"]

In [ ]:
audio_file

In [ ]:
classifier = pipeline("audio-classification", model="stevhliu/my_awesome_minds_model")


In [ ]:
classifier(audio_file)


In [ ]:
IPython.display.display(IPython.display.Audio(audio_file, rate=22050, autoplay=True))